In [18]:
import os
import time
from threading import Thread

from jupyter_core.paths import jupyter_data_dir
import notebook
import IPython
from IPython.display import display, Javascript

import tornado.httpserver
import tornado.websocket
import tornado.ioloop
import tornado.web
import socket
import json
import asyncio
import logging

def find_free_port():
    s = socket.socket()
    s.bind(('',0)) # find an available port
    return s.getsockname()[1]

__SOCKET_PORT = find_free_port()

wsConnected = False

class WSHandler(tornado.websocket.WebSocketHandler):
    def open(self):
        global wsConnected
        wsConnected = True
        print("Websocket Open")

    def on_message(self, message):
        #ws_queue.put(message)
        pass

    def on_close(self):
        print("Close Websocket")
        self.stop_tornado()

    def stop_tornado(self):
        ioloop = tornado.ioloop.IOLoop.instance()
        ioloop.add_callback(ioloop.stop)

    def check_origin(self, origin):
        return True

def start_server():
    asyncio.set_event_loop(asyncio.new_event_loop())
    application = tornado.web.Application([(r'/ws', WSHandler),])
    http_server = tornado.httpserver.HTTPServer(application)
    http_server.listen(__SOCKET_PORT)
    Log = logging.getLogger('tornado.access')
    level = logging.getLevelName('WARN')
    Log.setLevel(level)
    tornado.ioloop.IOLoop.instance().start()


# Removed check for ipykernel version because the old check
# was for 5.0.0 but this works with 4.x too...and 4.x is the first
# version of ipykernel
t = Thread(target=start_server, args=())
t.start()


In [19]:
__SOCKET_PORT

50107

In [23]:
%%javascript
var ws = null
var isopen = false
var intervalID = null

        // create websocket instance
        var port = 50107
        var uri = '/ws'
        var loc = document.location, new_uri, url;
        if (loc.protocol === "https:") {
            new_uri = "wss:";
        } else {
            new_uri = "ws:";
        }
        if (document.location.hostname.includes("localhost")){
            new_uri += '//' + document.location.hostname + ':' + port + uri;
            url = new_uri;
        }
        else {
            new_uri += '//' + document.location.hostname + document.location.pathname + uri;
            url = new_uri.replace("notebooks", "proxy/"+port);
        }
        ws = new WebSocket(url);

        ws.binaryType = "arraybuffer";

        // Handle incoming websocket message callback
        ws.onmessage = function(evt) {
            console.log("WebSocket Message Received: " + evt.data)
        };
 
        // Close Websocket callback
        ws.onclose = function(evt) {
            ws = null
            isopen = false
            console.log("onclose intervalID = ",intervalID)
            clearInterval(intervalID);
            console.log("***WebSocket Connection Closed***");
        };
 
        // Open Websocket callback
        ws.onopen = function(evt) {
            isopen = true
            console.log("***WebSocket Connection Opened***");
            intervalID = setInterval(function(){if (ws.readyState === WebSocket.OPEN) {ws.send("Hello World");console.log("Sent 'Hello World' to websocket");}else{console.log("Websocket is not OPEN")}}, 5000);
            console.log("open intervalID = ",intervalID)
            setTimeout(function(){ ws.close();console.log("Closing Websocket"); }, 30000);

        };

        ws.onerror = function(event) {
            console.error("WebSocket error observed:", event);
        };


<IPython.core.display.Javascript object>

In [21]:
t.is_alive()

True

Close Websocket
